# 2. Q-Learning

## 2.1 Learning from experts

In [2]:
from tic_env import TictactoeEnv
from Q_learning import eps_policy

env = TictactoeEnv()

In [3]:
Q_table = eps_policy(env, print_last_10_games=True)

-------------------------------------------
Game 19990 ended, winner is player X
Optimal player = O
Agent player = X
|X X O|
|X O -|
|X - O|

-------------------------------------------
Game 19991 ended, winner is player None
Optimal player = X
Agent player = O
|O X O|
|X O X|
|X O X|

-------------------------------------------
Game 19992 ended, winner is player X
Optimal player = O
Agent player = X
|X X X|
|O O -|
|- - -|

-------------------------------------------
Game 19993 ended, winner is player None
Optimal player = X
Agent player = O
|O X O|
|X O X|
|X O X|

-------------------------------------------
Game 19994 ended, winner is player X
Optimal player = O
Agent player = X
|X X O|
|X O O|
|X - -|

-------------------------------------------
Game 19995 ended, winner is player None
Optimal player = X
Agent player = O
|O X X|
|X O O|
|X O X|

-------------------------------------------
Game 19996 ended, winner is player X
Optimal player = O
Agent player = X
|X X X|
|- O -|
|- O -